# 1.2.8

In [4]:
def HammingDistance(p, q):
    distance = 0
    for i, c1 in enumerate(p):
        if c1 != q[i]:
            distance = distance + 1
    return distance

def Neighbors(Pattern, d):
    if d == 0:
        return set([Pattern])
    if len(Pattern) == 1:
        return set(['A', 'C', 'G', 'T'])
    Neighborhood = set()
    Suffix = Pattern[1:]
    SuffixNeighbors = Neighbors(Suffix, d)
    for Text in SuffixNeighbors:
        if HammingDistance(Suffix, Text) < d:
            for x in 'ACGT':
                Neighborhood.add(x + Text)
        else:
            Neighborhood.add(Pattern[0] + Text)
    return Neighborhood

def kmers(text, k):
    patterns = set()
    for i in range(len(text) - k + 1):
        patterns.add(text[i: i+k])
    return patterns
    
def MotifEnumeration(dna, k, d):
    common_patterns = {}
    dna = dna.split('\n')
    dna_kmers = set()
    kmers_list = []
    for dna_line in dna:
        line_kmers = kmers(dna_line, k)
        kmers_list.append(line_kmers)
        for kmer_ in line_kmers:
            dna_kmers = dna_kmers.union(Neighbors(kmer_, d))
        
    for pattern in dna_kmers:
        #print('pattern:', pattern)
        for line_number, line_kmers in enumerate(kmers_list):
            #print(line_kmers, line_number)
            for pattern_prime in line_kmers:
                if HammingDistance(pattern_prime, pattern) <= d:
                    #print(pattern_prime)
                    if pattern not in common_patterns:
                        common_patterns[pattern] = set()
                    common_patterns[pattern].add(line_number)
                    
                    #if pattern_prime not in common_patterns:
                    #    common_patterns[pattern_prime] = set()
                    #common_patterns[pattern_prime].add(line_number)
        #print('_____')
        final_patterns = []
        for k, v in common_patterns.items():
            if len(v) == len(dna):
                final_patterns.append(k)
    return ' '.join(final_patterns)

In [7]:
dna = """AGTCGTAACAGGGAAGGCAAAATTT
TTGTTGGCAAATAGTAAATTGTTCA
CAAGCGCTTAGGCAACCGGCCTTCG
TCCCAGAACGGGCGAGTCGCATTGA
CGACGATTTTAATCAATTAGGGCTA
CGGTGATCTGGGCGAGAAGTCCGAC"""
%time print(MotifEnumeration(dna, 5, 1))

AGTCA GGCAA GGCTA GGGCA GGCCA GGCGA
CPU times: user 193 ms, sys: 3.08 ms, total: 196 ms
Wall time: 208 ms


In [ ]:
# Este approach no funciono
def MotifEnumeration(dna, k, d):
    dna = dna.split('\n')
    # Patrones de la primera fila
    common_patterns = kmers(dna[0], k)
    print(0, dna[0], common_patterns)
    for i in range(len(dna) - 2):
        patterns_prime = kmers(dna[i + 1], k)
        print(i+1, dna[i + 1], patterns_prime)
        new_common_patterns = set()
        #print(common_patterns)
        for pattern in common_patterns:
            for pattern_prime in patterns_prime:
                if HammingDistance(pattern_prime, pattern) <= d:
                    new_common_patterns.add(pattern_prime)
                    new_common_patterns.add(pattern)
                    # print(new_common_patterns)
        common_patterns = new_common_patterns
        print('common patters:', common_patterns)
    return common_patterns

# Motif entropy

In [60]:
import numpy as np
from collections import Counter
def motif_entropy(motifs):
    motifs_matrix = []
    for motif in motifs:
        row = []
        for c in motif:
            row.append(c)
        motifs_matrix.append(row)
    motifs_matrix = np.array(motifs_matrix)
    length, width = motifs_matrix.shape
    total_entropy = 0
    for i in range(width):
        frecs = Counter(motifs_matrix[:,i])
        entropy = 0
        for k, v in frecs.items():
            p = v/length
            entropy = entropy - p*np.log(p)/np.log(2)
        total_entropy = total_entropy + entropy
    return total_entropy

In [61]:
motifs = ["TCGGGGGTTTTT",
"CCGGTGACTTAC",
"ACGGGGATTTTC",
"TTGGGGACTTTT",
"ATGGGGACTTCC",
"TCGGGGACTTCC",
"TCGGGGATTCAT",
"TAGGGGATTCCT",
"TAGGGGAACTAC",
"TCGGGTATAACC"]

In [62]:
motif_entropy(motifs)

9.916290005356974

# Median String

In [63]:
from itertools import product

In [357]:
from itertools import product
def HammingDistance(p, q):
    distance = 0
    for i, c1 in enumerate(p):
        if c1 != q[i]:
            distance = distance + 1
    return distance

def kmers(text, k):
    patterns = set()
    for i in range(len(text) - k + 1):
        patterns.add(text[i: i+k])
    return patterns

def MedianString(dna, k):
    distance = (k + 1) * len(dna)
    for kmer in product('ACGT', repeat=k):
        kmer = ''.join(kmer)
        accumulated_distances = 0
        for dna_line in dna:
            line_min_distance = k + 1
            kmers_line = kmers(dna_line, k)
            for kmer_ in kmers_line:
                d = HammingDistance(kmer_, kmer)
                if line_min_distance>d:
                    line_min_distance = d
            # En line_min_distance esta la menor distancia de kmer contra la linea
            accumulated_distances = accumulated_distances + line_min_distance
        if accumulated_distances < distance:
            distance = accumulated_distances
            motif = kmer    
    return motif

In [351]:
dna = """CACATTAACTGAATAAATGGGGAGAATGACTCTGCACGGGTG
TGTCGGGGGAAGTGGACTAGCCAAAATGTTAAACATTCGCAT
GGGCAGCTCATACCGGGGGCACGCGAGATCGGGGCTTTAGTA
GGGTAGTCTTGCTTGCCTGATATATGGCGTCCACGAGTAGCG
CGGCCGGCTGGCTAGGGGATAGTAGAAAATAACAGGGGGGAG
AATAGGTCCCCCGGGCAGAGCATATATATTAAAGGGCGATCC
AATCTAGTAAAGAACTCCCCTTGTGGGCAGGCAGATTGCTAG
GGGAAGCTATACCCGGCTCAGGATGATACGATCACTCCTACA
CGGTGAAGTTACATATCTCAAAACGGGCAGAGGCAGCGTCTT
TCATCAGCAATGTTAGAACGCGTAGGGAAGACGTGCCGTCTT"""
dna = dna.split('\n')
MedianString(dna, 6)

'GGGCAG'

In [353]:
dna = """CTCGATGAGTAGGAAAGTAGTTTCACTGGGCGAACCACCCCGGCGCTAATCCTAGTGCCC
GCAATCCTACCCGAGGCCACATATCAGTAGGAACTAGAACCACCACGGGTGGCTAGTTTC
GGTGTTGAACCACGGGGTTAGTTTCATCTATTGTAGGAATCGGCTTCAAATCCTACACAG"""
dna = dna.split('\n')
MedianString(dna, 7)

'AATCCTA'

In [379]:
motifs = ['GTCAGCG',
'AACGCTG',
'AATCCTA',
'GTAGGAA',
'TCTGAAG',
'GATGAGT']
for d in dna:
    dist = len(motifs[0])
    for km in kmers(d, 7):
        h = HammingDistance(km, motifs[0])
        if dist > h:
            dist = h
    print(dist)

3
3
3


In [166]:
def HammingDistance(p, q):
    distance = 0
    for i, c1 in enumerate(p):
        if c1 != q[i]:
            distance = distance + 1
    return distance

def kmers(text, k):
    patterns = set()
    for i in range(len(text) - k + 1):
        patterns.add(text[i: i+k])
    return patterns

def DistanceBetweenPatternAndStrings(Pattern, dna):
    k = len(Pattern)
    accumulated_distances = 0
    for dna_line in dna:
        line_min_distance = k + 1
        kmers_line = kmers(dna_line, k)
        for kmer_ in kmers_line:
            d = HammingDistance(kmer_, Pattern)
            if line_min_distance>d:
                line_min_distance = d
        # En line_min_distance esta la menor distancia de kmer contra la linea
        accumulated_distances = accumulated_distances + line_min_distance
    return accumulated_distances

In [172]:
dna = 'GATTAGAGGTGTGACTGGCCGCCGGCGTAGGCGGGTTTTACCTTTTGTAATGTGATCTAAACATTGGTGCCACACGGCTCGCAGGTAGATAA GATTCGCATGGGGGAGCCACGTGGGCTTAAGCGCGGTGGCCCCGTTCTGTTTGTGCTATACTCCCCAAGTCTTTAACACACTGTCATTGCAA TTAAGTTGTCTTCGATCTACGTTCCTCGGTATACGGCCCGCATGTTGACATATTACTGTAACTTAGATGTGATAGGATTCACTCCTACCCTA CAGTATCAGCATAGTTCCTAGATCCATGGAGTTCGAGATGCGGGTTTGTAGCTCAGAGGACGTTTGGGTCGGAGATGTAGACCGCGTGCACC TGTCAAGATGGGTGTTCCGGAGAGAAGCGTAGCCAAGGAAGTGTACCTCTATAACGATATGGGGATACCTTCACAGGCGCAACGATGAAGAT GTTATGAGGGACATGATCAACGCGGGCGAGCCAAGTGTGATTTCATGCTCTAGCTGTACTGAGGTACGCTCCCACTTATGTAACCTGATCCC AGGTATGCATTGTTCGATCACTGCAAAACACGCTGCCTAATTTGGGAATTTCTGTTCGCTTTATGGAGGCCAGCGTACACCGACAGGCCCAG CCAGCCACCTCTGCTACGGTTACACAGATTTTATAGCAGTGTCAACTTTCTATGTATCCCGGGAGGGGCCTACCTTCGTAAACAGACCAGGG TTCATGTGATCGTTGTCGAGCTACCCAGGATCGCGTGCTATAACACGTACCGATCCTCGCATCTGCCAAGACCCCCCTTTCAGGTCTCGGCT GGAGGAATTACGGCACCTCTACTCTGACAAGAGTTCCTATCTGCGTGCGAGAACATTAAGTAGGGCCGACCGCTAACAATACGTGGAAGAGG ACAGACCGTAAATTAAGGTAGTGAATGTCTACGGAGCAAATGAAATACAATCTTGTCCGCCCCTCGCCGTCGAATTATGCGCCAGAAAGCTG CTAACGTACGCGTACACACTTCTGGTGACTCCTGGCAGGATGCTGTGTAATACTCACCTTAGTCCACCTAAATCGCTTCCTCACCTAACGTG GTGGTGCACCGAGAATTATCTTACTACATTCCAGGTATGTCAGCATGACGTACCTCACTTGTATAAACTCGCAACCTTCGGCCCTTCACATG TACAGTTATTTGCCGGTTTATCCGGCACCATGCTGAGTCGATGCACGCGTGACGACAAGAGAACAACTCTTCCGACTTCCAGCGGGCGTGAG GATTTGGGCGAAATCACCCAACCAACCACTCTCAGGAGAGTAGGCTGGGAGAAGTGTTCAGTATTCGGCGGGATCACCTCAGTCTCAATATA CAATGTCATGGCGAGTGTCGGTTGTCTATACCAAGTCTGGAGCGAGCCCAGAACGCGCAGTACCCCGCGTCGGAGTATTAGCTGTTCAGCCG TGCAACTAGGCGAAAGCTCGGCCGTATAAGTAACAGATTGGACCATTAGCTAGGGGTGCTAACCCCCCAGAGCCTCGACTAGACGCTAAGCG GGCCTACACGATTACCCTACTGGGCCAGCTATTTGGGGCTGCTGGTGGATATTCACTTCGCGAGCTAATGTCAGGCCTATCGCGTTGTTGAT TCGGACGACCCGTCACCTGTGTAGGGTGAGCGCGGGAGCATTCTTGCTCACGCCCACTTGTAGAGCCTTATGGTCCGTTACCACGCAGTTTG ATTCCGAGTGTCCCTGATCAGGTGTAATCATGTGATGGGACCTGGAGTAGCATGGGGCGTGATGTGTATTTTTGCGAGACGCCTCCACACAC CACGCAATAGACTGCGTTAGCGAAATGCATCCACCGGTACCCCGAACGCTAACTCATGATAATAGTCCACAACCCTGACGGTCCGTTACCTC ACAGGCATAATGATGGGGCAAGATTCGGCGGTTTAATGTTCAGAGGTTAAAGGCTCTAGTTGCAAAGGCTAGTGCTCACCCAGAGCCACGGC GATTTATGCCGAGCTACGCACCGTTCTACGAGCCGGCGATTACTTACCGTTACTGATTCGCAGACCCCAAAAAAAACCGATCGAGAGCTTTT GACGCATGACACATGTTTATGAAAACGCATCTTACTTCCATCATGGCAAGTTGTAACGGACCGACGCCCATTGGTTAGTGAAGTATCTGAGT TTAAGACAAGACAACGATGCCAGTAAACTCAGGTTAGGAGACTGTCTGTGCGGAATAATGTACGACCTTAGACGCGTGAGAGCTCTTTATCA TGATCACGACCAACCTCCATTTCAACGTTGATAACGGTTGATATCTAAAAGTGCCACGGACACCAGTACCAAGACGTCATGCTCTGAAGATA GGCAACCCAGGTGTCGAGAGAAATTGTAATAGGCAGTTGTGCGAAAGCCTAAGCCCAGGGTGGCATGTGCTGAGATGATGTGTGTTTATGGG TCCGCCACCTGACATTTCTCAGTATACGAACACGACTGGTGTTACAGTTCCTATCTTGCCAACCGACGGATACCTGGAAGGAGGGCGGGAAT TTATAGCGTCGCAGCAGCTCTTCCATCCCCAGAGCGTCAGACTATTTCCTAATCGGCTTAAAAAACCCTCAGACGATGGTATTGAGAGGGTT GACTTGCTTTATCGGGAGCGGTTCGGCTCAGCGCTGAATGGTCGGTTCGCTTTAACAGCCGGATAATCAGTTCACCATAGGGACGTAGGGCA ACATGCGATTGGGATTGCCCATAGGGCACCCGTCTGCGGCTAGAATAGTGTACTAAATTGCCACTGCTCCGATCCCGAAAGTGGGAGAGTAC GGAGGGAGGACTCCGGTTCTACCAGTATGCCGCATTCTAAGGGGAGCTCGGCTCTAGGGAACGGCCAGTCCCGTATGGGCGTCACGGGACGA ACTACTGGCGTCCCTACCGTTCCTTGTCGCCATGTATCACTTCTCTGTAGTAGGAAGTCACTTGGTGAAACGATAGTGCCATCGTGTGTCCA GGCGGTCTTTCCTCGATAATTTGCGGGAAGGTCACACTTAGAAACTAAAACCCATAGGTCGAGTTACAGAGCCAGGTTTTACGCTCATTATA AGCCGGACGTGAACCATGCCGGTGATGTATGCGCGCATTGTAAGACGGCCCAGCCTGTTAAGTGGACGTTCAGTACAGAGCTTCCGGAGGTC ACGTCGCGTCAACTGCTTTAACTCTCGTGATAAAAGGAGGCTAAAATAAGTTTGTCTACGCCGGAGGACTTGGCCCAGGGGTACTAGGGACC GCATCGCTCACGTAAGGGAATTCAAGACAACCAAAGTTGCCGTAAGACCATACTCGCAGGGTTTTTGGCTCACCTGTTTCGTCTAGATTCTC'.split(' ')
Pattern = 'CGGGCT'

DistanceBetweenPatternAndStrings(Pattern, dna)

65

# probability of kmers

In [338]:
from itertools import product
def kmers(text, k):
    patterns = set()
    for i in range(len(text) - k + 1):
        patterns.add(text[i: i+k])
    return patterns

def getProb(km, profile):
    p = 1
    for i, c in enumerate(km):
        p = p*profile[c][i]
    return p

def ProfileMostProbableKmer(text, k, profile):
    kms = kmers(text, k)
    prob = 0
    for km in kms:
        p = getProb(km, profile)
        if p > prob:
            prob = p
            motif = km
    return motif

In [129]:
text = 'CTACAGATGTGGATTGACCCTCATGGCCTAGCCCTTGGCAGACGAGATTATGTGGCTCTTCGGGAAGCGCGGAAAAAAACTCCAACTGAAACGGTAAGCAGATAGGCTCCCCCCGTGATTCCGAGCCTAACGTATGCCGGGAGCCTGTTGCTGCCTCCATGTTTCTCGTTTATTTCCGATAACACAGATGGCCGAAAGTGGAAGTATCGTAATCCGTCGTTACCTTAGCTCTAACGTTAGATACTATCCAGCCATAGAGACAGTAGTTGATGTGATTGCTAACTCAAGCGCGGAATTCGCGGTTCTATCATCCTGACTCGAAGTCAAATAACGACTTAATGCACGTACTCCTGGTTTGAGGTTCAAATTGACAGGAAAGCCGACGCAATTAGCGCAAACCCACGTAAAAGTTTTACACGACGTTATGGCTCAATTACAACCATAGTGTGGCCCGGAGTTGCGAGCACTTAATTGGGGTTTAGGGGGACTGCGCAGGCAAGTTGGGTTAGCGAATGAGCCACCCATCCGGTGCAAAAAGTTCCTTATGAAAAAGATGATGGTCTCTGCAGAGCACGAAGCTGCGCGAGCAAGCTGTTGTGTGGTTTTTGCGGTCTACCCAACCCTATCTACGCAGTCTTACTGAGTGAAACTTAGGTCTAGTCTGACATCATCCCTAGGGTCTCGTTGCCTTAGATATTAGATCAGAACTATATCAAGGCTGGCAAGGTTGGGTACCCCATAATTCATCAGACTTTGTAACAAAATCCTAAGTCACTCTAAAGCGCGAAGTATTCTAGTTTCTTCTCTAGAACTTACGGATTAAGTGTACTCAGGCTCACATGTAGCTAGTTTTCTGAGCGTTACAACGAATCTCTGGGAGGGAGTACGATTCCACTTCGTAATCTACCAATATGATGGCTATGGCGTGTATCACTGGTGATTGCGCAACCCTACTGCTCTAGAGTGCCCGCTGTTCTGAGACGCATGATTCACCAAAGGT'
k = 14
profile = {
    'A': [0.197, 0.296, 0.141, 0.310, 0.225, 0.296, 0.380, 0.239, 0.225, 0.169, 0.225, 0.211, 0.225, 0.324],
    'C': [0.268, 0.197, 0.254, 0.211, 0.254, 0.197, 0.296, 0.296, 0.239, 0.296, 0.282, 0.282, 0.296, 0.211],
    'G': [0.254, 0.225, 0.338, 0.310, 0.239, 0.324, 0.085, 0.239, 0.254, 0.296, 0.310, 0.239, 0.282, 0.282],
    'T': [0.282, 0.282, 0.268, 0.169, 0.282, 0.183, 0.239, 0.225, 0.282, 0.239, 0.183, 0.268, 0.197, 0.183]
}
ProfileMostProbableKmer(text, k, profile)

'CTGGTGATTGCGCA'

# GreedyMotifSearch

In [272]:
def kmers(text, k):
    patterns = []
    for i in range(len(text) - k + 1):
        patterns.append(text[i: i+k])
    return patterns

def get_profile(motifs):
    motifs_matrix = []
    for motif in motifs:
        row = []
        for c in motif:
            row.append(c)
        motifs_matrix.append(row)
    motifs_matrix = np.array(motifs_matrix)
    height, width = motifs_matrix.shape
    profile = {'A': [], 'C': [], 'G': [], 'T': []}
    for i in range(width):
        frecs = dict(Counter(motifs_matrix[:,i]))
        for k in profile.keys():
            if k in frecs:
                profile[k].append(frecs[k]/height)
            else:
                profile[k].append(0)
    return profile

def getProb(km, profile):
    p = 1
    for i, c in enumerate(km):
        p = p*profile[c][i]
    return p

def ProfileMostProbableKmer(text, k, profile):
    kms = kmers(text, k)
    prob = 0
    for km in kms:
        p = getProb(km, profile)
        if p > prob:
            prob = p
            motif = km
    if prob == 0:
        motif = text[:k]
    return motif

def score(motifs):
    m = []
    l = []
    for k,v in get_profile(motifs).items():
        l.append(k)
        m.append(v)
    amax = np.argmax(np.array(m), axis = 0)
    motif = ''.join(np.array(l)[amax])
    score = 0
    for m in motifs:
        score = score + HammingDistance(m,motif)
    return score

def GreedyMotifSearch(Dna, k, t):
    BestMotifs = []
    for line in Dna:
        BestMotifs.append(line[:k])
    line = Dna[0]
    for j in range(len(line) - k + 1):
        Motif1 = line[j: j + k]
        Motifs = [Motif1]
        for i in range(t-1):
            profile = get_profile(Motifs)
            Motifi = ProfileMostProbableKmer(Dna[i+1], k, profile)
            Motifs.append(Motifi)
        if score(Motifs) < score(BestMotifs):
            BestMotifs = Motifs
    return '\n'.join(BestMotifs)

In [275]:
Dna = """TGGAGTCCCTGTACCCCATGGATCCATCTGGCTCACACTAATCATTCGCACTGAGTAGTGACGGACCTTCTGAAAAGATAGCACAAAAGCGTAGTTGTTATTGCTACTTGAACAGCGTCCTCAGGACACGCCTTAACCCACTCTTGAAAATATGGG
TTGCCGTTCCGAATAGATGGGAGCGGCGTATTGGTGGCATGTCACGATCAATTGGGTTCGGTAAGTTAGGCAGTCTTGGGGAATTCCTAGTGTGTTCTGCTGGGACGACTTTGCTTACAGACCGTAGGCATCAGTTGGGAGAGGAACATCTGAAAC
ACACACGAATGTAGTGCACTAATGACCCGAAGTATCGAGCTCAGGGGGAATACATTCGGTCTCTCATCCTACACAATTAGGAGTATGTCCAACCCGGCATACAGTGAATGGGTCATACAGTAGTATTGCCGATATAATCGCGGGTTGTTGGCATGC
ACCTCTTCCTGGTTTAGATGTCTCAGTACTTACGAGTGGGAGTCCACAACCCTACGAATCAAATGAGGTGGTACCCAGCAAAATCATAGAAGTAAACTCTGTAATCACTAACGGATCACCTCACTGCCATTACGAAAACATCAGCAGAGTACGTCA
TCCTAAACAGAGGGAAGTGCTGCTTAACCAAGCGATGAGAGCCCTTTTAGTGACGGCGGTACCGTACGGATCGTCTGGTTCGGTAAAGCCCACACTCAAGGTCATTATTCGATAAAAGTAGCCTAAGTTAAGCATCAGCCTAGATAAATGTTCGCA
CTAAAGGATTCAATAAACCCTTTTCAGACGAATGGCTTATCCTCTGTAGCGGGGTGTGCGCTAGTCCAATCGTGTACCGAGACGCTGCTTATATTAGGTGCATATCGGACCTGAGGTATCTTCGAAGTAATCAGCGAACTGGCTGTTAAGCTCAGA
CAAGCTCCAACTCCGCTGGAGCCTCAGCTCCCCAATGAATGCAATGGGCAGACGTACACCCAGCTTCATAGGCATCCAATTTGCACCTGATGAATCTTTGCTTTGGTCCCTCTAATCTTTGCATGTTCCGCTGAACAGCCCTCTCGACAACGTCGC
TTCGACCAACTAGTGTAGGGGAGTAAAAGCCAATTACAGTCGTATCTTCATTATTGTGCTCCTCTTATGTCGTATCGCTCAAATGCATCGTGACCCAATATAATAGACGATGCTCTAAGCATAGTTAGTTTCACCCAAAGAATCGTTCTTACGCCG
CCTGTACTGTGCACTAATATACTCTTGCAGCTGTTACCGACTCGGCAGTCAGTTCTAGCGTTCGGGATAACCTCTCGAGGCACTCATAAACCCGGATGCTGCCACTTGACGCTCGCTGTTACCCGAGGAATCTTAAGACACTGCGATCCTAGCATC
CACATGAGCCGTACGGATTGCCCTAGGGGAGACAGGGACCGGGTCGGCGGGGACGTGTCCTCCCTCGTTTCATCGTAATCTCCCACCGGAAGAATCTTAGTCACACCCGCTTGGTAGCGTAAATGGATGACGTACTGGATAGTTTGGTCTATTGAC
GCTGGTCACCGGACTGGTGAGCACCTATTGGACGAGCCGTTAGCGTGTCGAAGATCAGCGGTATGGGTAGGCAGGGTGTCGCAAGGGAAACTTGGGACCAAACGGATCTTACACGTGGGTGACGACTTGAGCTACAGTACTGGATGCCCCTTTAGT
AGCACCCAGCGACTGACATTTTCTCAGGTACGGTATTCGGATGTTCCACTAGAACCTAGCGAAAGCCATTGGTACGTCGGCGTGATTGCCCCCGTTGTACGAAACATAACCGGAAGCATCCGACAATCAGACTGTGGATGGGGAGGCGTTCTCAAC
GATGTGACTAATCATAGAAACTTAAGTGCCTTAATGGGGCGTCACTACGTTACCGACCGGTTTCTCCGTGCACCAAACCCCTCTTACGCCGACAGGCTTGACCAGTGACACTGTACCGCCACCAGACGGATCGTGGGTGCTGTCGGGGAGATCTGC
ATCCGAACAAAGCTTCAACGGGTTGTGTACTTCCGTATGAGAGGGAACAAGATCGTCCTCACATATAAGCTAGAGCCCTTCTAATATCGCACTGGAACCGCATGGATCTTGGAATGCCAACCACACCTAGTCGGCAGCAGTCGGTTATGGGCACTA
GTTTAACCCTGGAACAATCGCCTGAGTCGCCGCGAAATGACTGCCATGTGGAATGGATGGCCTCTGGTTTAATCTCCGGTCTAGTGCACATCTTGTTATCAACTTCCTACCCCATGAATCACACAGTGACCTGCTGTAGATGTTCTTAGCGCACGA
CGGTGAAACTTACGACCTAAAGTAAAGAAATTGTGCGGGAACTCTGGGGACTACGCGGCGGGCGTGCGCGCCTAGTTATTGGCGGGTGGCTTCGCTACCACAGGGATCCCCAAATTCGCACATTTTCTATAAAGAAATCACGTCGGAGGTTTTTAA
CGTGTTCTCGCGCGATAAGGTCTGGATATACAACCAGACAGTCGCGAGGGTGATTAATCATTTTCACTTCCACCAAGTATTTACGGCCCTAACAAGAAGAAAGTCAGGTAGTGTGGACAGACCCCAGGCATCAACATACTTTATGACTCGGATGAG
CACACTCTGTCATGTCGGACTAGGTGTGCAGCTGGGTCCTCCATATCGCTGACACAGATTCGAAAGTGGCTCTCCGAGATGACGAAAGTATTGCAGGGCACCTTTACTACCACACGCATCAGCGAGATTAAGCGGGCACATTGTTTGAACTGGCTG
AAACAATGAGTGAAAGCGCGTTCGGGGACAGGTACCGCCCTCTTCTCGCAGAATTCTTAGATATCTGAGGCATCCAGCCAACGCGCATGTGTGGGAACCCCACGAATCGAAAGAATCTTAGAATCCGAATACAATGTAGTCGGGGGACGTACGGCT
GAATGAATGAGAAACTCGTTTATAACCGCACTGAACTGTCTCAAGCATTGTTATACACAAAAACGCGTCGTGTTCCTATTGACACATACAAAATAAATTTCGGTCTCCTATTCGTTCAAACTATGAAGAAGCACCCTAAGCATCATTCCTACAAGT
TCAAGCCATTATGTACTGCGTACCGTTCGAGACCGGACGACGGACGTAGTGTGTTCCTAGATACGCCAGGGCAATCGCTACATGACGGGGTAGCTTGCACTGAGACTCCCACTAACTTCAACCCAATGTATCAATCATTTGATAGCATCTACATCG
CAAAGGGATTTGCATAGGTACAAGGTTCACGCCCTTAGACTATGAGGAGGGAAGACCCCCAGTAGCGTGGAAAGTATCTGCGGTTTTTGGCCGGGTCATATTGGGTTCCATTCATGCAAGGCCGCGCCCGTTATATCTTAAAATACCCGATGAATC
TTGGGACTGCGAAGCGTGTTGCGGGTTTGATACCGCTGCATCGGGCTGACCGAAAGGATCCTGGAAGGAGACTCGCTGGTATAGTAGGATATATCTTTTTTCGATTCGCCTTTCGGCTCAGTTTAGAACCCCGTAGGACTCCAATTACTACGCCTC
GAGAGAAGCCGTTAGGCTTAAGAAAGATCTGTTTAGGGTACGATGGTCCGGATATCCAGAAGTGCCACACTCGGGTCACGCCGAACGCACTAGGGGTGAACTCCGGTGACGGAAAGGCAGTGACAAGCCTACACCTAAAGGATCCGCAAGACCCCG
CCTAAGGGACCTGGGTGACTTGTATAAATTCCTCCCACCCGCCTGATTTGTTGTCAGGTAGGGTAAGTCCGAGCAGCTTCCTATGCTCACGGTCGATACAATTGTCTCATTGTGCACCCAACCTGAAGAATCCCAAATTAACGATACCGGGAGACA"""
Dna = Dna.split('\n')
k = 12
t = len(Dna)
print(t)
print(GreedyMotifSearch(Dna, k, t))

25
CCCCATGGATCC
TTGCCGTTCCGA
ACACACGAATGT
ACCTCTTCCTGG
CCGTACGGATCG
CTAAAGGATTCA
CCCAATGAATGC
TCAAATGCATCG
ATATACTCTTGC
ACACCCGCTTGG
CCAAACGGATCT
AGCACCCAGCGA
CCAAACCCCTCT
CCGCATGGATCT
CCCCATGAATCA
CCACAGGGATCC
CCCCAGGCATCA
CCACACGCATCA
CCCCACGAATCG
CTCAAGCATTGT
CCCAATGTATCA
TTCCATTCATGC
CCGTAGGACTCC
CTAAAGGATCCG
CCACCCGCCTGA


# With laplace

In [334]:
import numpy as np
from collections import Counter

def HammingDistance(p, q):
    distance = 0
    for i, c1 in enumerate(p):
        if c1 != q[i]:
            distance = distance + 1
    return distance

def kmers(text, k):
    patterns = []
    for i in range(len(text) - k + 1):
        patterns.append(text[i: i+k])
    return patterns

def get_profile_laplace(motifs):
    motifs_matrix = []
    klen = len(motifs[0])
    for motif in motifs:
        row = []
        for c in motif:
            row.append(c)
        motifs_matrix.append(row)
    motifs_matrix = np.array(motifs_matrix)
    height, width = motifs_matrix.shape
    profile = {'A': [], 'C': [], 'G': [], 'T': []}
    for i in range(width):
        frecs = dict(Counter(motifs_matrix[:,i]))
        for k in profile.keys():
            if k in frecs:
                profile[k].append((frecs[k] + 1)/(height + klen))
            else:
                profile[k].append(1 / (height + klen))
    return profile

def getProb(km, profile):
    p = 1
    for i, c in enumerate(km):
        p = p*profile[c][i]
    return p

def ProfileMostProbableKmer(text, k, profile):
    kms = kmers(text, k)
    prob = 0
    for km in kms:
        p = getProb(km, profile)
        if p > prob:
            prob = p
            motif = km
    if prob == 0:
        motif = text[:k]
    return motif

def get_score(motifs):
    #Obtiene el mejor y calcula diferencia
    m = []
    l = []
    for k,v in get_profile_laplace(motifs).items():
        l.append(k)
        m.append(v)
    amax = np.argmax(np.array(m), axis = 0)
    motif = ''.join(np.array(l)[amax])
    score = 0
    for m in motifs:
        score = score + HammingDistance(m,motif)
    return score

def GreedyMotifSearch(Dna, k, t):
    BestMotifs = []
    for line in Dna:
        BestMotifs.append(line[:k])
    BestScore = get_score(BestMotifs)
    line = Dna[0]
    for j in range(len(line) - k + 1):
        Motif1 = line[j: j + k]
        Motifs = [Motif1]
        for i in range(t-1):
            profile = get_profile_laplace(Motifs)
            Motifi = ProfileMostProbableKmer(Dna[i+1], k, profile)
            Motifs.append(Motifi)
        motScore = get_score(Motifs)
        if motScore < get_score(BestMotifs):
            BestMotifs = Motifs
            BestScore = motScore
    return '\n'.join(BestMotifs), BestScore

In [336]:
Dna = """GACCCATTACCCAGATCTAAGGCCCGCGAGCGGACGGCCCGGCGACCGATGAATTCATGTGTAGTCGCCGTATGTGCTCGGCCTCCCTGTTGTGGATGCGATGTAAGACAGGCTACGACATCTATTTAAGCGGTGAGGTTATGATTATCTCGCACC
GCGCTCGGCAGACAGCTTTAGGTTGTGGACATAGACCCCTGTTGGCTACACTACAGACACGGAACCCACATTATTCCGGATGGGGTTCTTTTGCTCTTAACGCAGCCTCATCGGTTAGGTCGGGGCCCCCTTCCACATTCCTGTGCACGATCCAAT
AAACCATCAACGCGACCATATACCACGTACTAGTTGAATTACCGCAAAGCACTGCTTGGGGTTGACGCGCGCCTCTACACCAAGGGCATTAAGTCCGGTTCCGATTTCCCCTGTTGAACCCTCGTTAATCGAGGTTTACCATATTGCAATTTTTAA
TCGCGGCGATGCTAATGAAGCCAGAAACGTACGGAACTCCGACTACGTCAAAATACTACCGAGCTGGCGGTTACGGATCCGGTACCCTGTTGGAGTTCACCAGTATGATGGAGGATATCCTCTATCTACCTAGGTTAGGGTCTAGGATAAGGTCAT
ATCATAAGATATATCAATATCGAACGCACCTCTTCCGCGATTAGCCAGAGGTTCCCGATCCATTGCGTGCCGGAACCGCGTCACGTACTATCGCCAAGCGTCGCTGGGCCCTGTTGTCACCTTCGGGTCAATACACCGTGATCCTGGTTGCCTAAG
GCCGATGATTCATCATACCTGTTTGTGCTTTTGTTGACCCCGCTCATCCGGTTACTATCCCCTCATCCTGATTCCTAAGTCCTTCGAGAGCTGAGGAATGTACAATCCCAACGTATGCGCGACTCGGCTGATCGAACACCGTAGCCCTGTTGCCTA
TGGTATAGAAGGGGGTGAGCTCGCCCCTGTTGGGTCGACTTGGTTGTAACCACTGATAACCGGTCTCCTATGCAACCGGACCTATCTTCGAGTAAAGTTATGAGATACTAACGATTGGATATTCACATTAGCGAGCCGGGCAATGTTACCCAGACT
ATAACCATGAATGAGTAGCGCCGGATTATAGGTTGTGCGGTTGGAGAGTCGTGGAGTGTCATCCCCTTTGACTGGTGCTATGGCCATGCCAGCTAATGGACGATCGGCCCCTGTTGATACGAACGTGTTCGCGTTTGCCCTTCGAAGTCTATTGCC
AGGCGGGGAAATCTGCTCAACACATGTTATTGTGTCATCTCGTCTTTCCGAGCTGCGATATGTCAACATCTTCTGCTTCACAAGTAAGTCTGACTGGAAATTTACAACAAACACCGGTCAAAACACTCCCTACCCTGTTGTATCAGCCGTATGGCA
CAAAACGAAGCATCCCTCACGTAAAAATGAGTCGCCCACACTCTTTCCGTATTTAAGATACAGCTTGTGCGCGACCGGATCCCCTGGGCCGGTGTCGAGTCGCCCTGGAAGGCATGAATCGGCGGGTTCTCACCCTGTTGTCTATACGGGAAGAGA
TGTACTTGCAACTCCGATTCTCGCGGCATTGTAATTACAAAGACACTCTCGCTCTTCCTCGCCTATGTAAGTTAGTAATCGCATTCACTGAATAATCCCTGTTGCCAGTTAATGTCGTCACACACCAAACACTCGATTGGTGTTAAACTGTTTGGT
ATTCATTACGATCAACTTAGAAATCCTACAAAACATGAACCAGTGACTGAAAAAATATATACTATATCCCATCCACCCCTTGCCGATAGCCGGCGCACCCCCGTATATTACTCAGAGCAATTTCCGTATTTTCACAGAGCCTGCCCCTGTTGAACC
GGTAACAACTGGATGATTGCCTCTACAGGGTCGTATACAATAGTCGACCGCATATGCTCCAGGGCTCCGTACCTTCCGTTTTTACCCTGTTGTTCTCGAAAGATAGGCTTGCCCATGCAATCCCAACACGAGAACAATAGATGTGTTCACTCATGA
GTCGTGCACGTGTCTCCGCAAGACTTGTACCACTTGTCTTTCTTGACGATATTTGATTGCCCCGTATACCGTCCCTGTTGTGGATATCGATCCGTCCGAAAAGAGCACTCAGCAATGGGACGGATGTAAGGATAGTAGACCGACTACTAGGAGGGT
AGCGAACTATTGCCCTGTTGTATAACTTTTACCTATCTCCATTAGCGGTTCTCGCAGAGGACTAAAAAATGGAGAAATTACACGGTAGCATGCGAAGCCGCATACACATCCGATATCTGGCCGGCGCCGAAACCTGATGGACTCACGCCTATATGA
CCCTGTTGCGAGATGGTTCTCCCGTAGATAAATGAACGGTATGGGGGCGACAGCGTCCGCCGTGCACGTTTTCCGCGGCAAACTTGGCGATGTCTTTAGGACCTGGGATCAGTGACACGGGGACGTTGACCGTATCTACTACAAAGCCGGCTTGCC
GCAGTATCCTTTCCCTGTTGCTGCTAGGTTCACGATGGCAACTTCTGGGCCACGGTCTGCTGGGTTCTACGTGATCGGCTCTCGAAGCGTGCAAGGAGGGCCGGGTATGACGCCAATCTTGATATAAAGCCTCAAAGGCGACCCCGACAGACCACT
TAATAATTTTGAATGAAGCGAACGCCCTGTTGCATCCTGGGGTGACCGAAGCCGAGCGAGTAACTCCAATCGATATAGGTGGTATGACACGCGAGATTCCACACCACCAAGTCCTTGATTTACTACATTAAGCCAGGACTAAATCTATTGAACATA
CAATCTCCTTTCTACATAGTAGATCCAAGACGAACTGATCTAGACACCCCCACTATCTATCCCTGTTGAGGTAGTATGGTGAAGTTAAGAGCCTTTAACTTAACATTCCGCCTTTTAAGAAAGAGCGTCACTATCGGGACACCCTAGGTGCTCCCA
TACTGGTGTCACCACATACTAAAAAGATAAAAATGTGTTTATAAAGCGTAGACGTCCGGTGTTTGCGTGAGGCAAGTGCACCTCAACAAGGACGCTTGTACGTCTCGGCGTATCACGCCCTAGCGGCCGCTCATCGCATCGAGCCCCTGTTGAACC
CGGATCCGTTGCAGACCGTACTCCCCCTGTTGCTTACTGTTCGTTCAAGGACTAGGGTTAAATAGGTTATACTTACGGCTTATTGTTTGCGTACGCGTCTGATCGCGCCGATAATGCGATGTCATTACGGACCAACATCACCCCTACATGAAAGAC
CCCTGTTGCTGGCACACGTTGCCTTGGGCACTGAAGCTGTCTATGCTTCTGGGACTGTTGCCGAGCGGGACACATGGTCACTTATAACTCGTGCTTATTGATCTCGTCCCTGTGGAATAACCCCAGTAGAACATACACAACGGTGCAAACGAAGTC
TCCTATTGAAGCGCGAACAACTCTGCCATAATATCGTGTCGCAAACTTTGTTAGCATGTTCGTGCACTGACACGCAAGACTGGTCTCTTCAGAGTGACCACACCTCTCCCCTGTTGGTAGTCACATAAACGTGTTGTTTGAGCGTTGTAACCTCGA
TCACTTGGCTGATGTCTCGCGGTTGGGTCTTTCTCAGTGATAGTGCGCCCCTGTTGCTCCCCGTTACTATCAAATATGTGAGCCACCACTATTTATGGCCGTTATCTTGATGATGCTACCACGACGTTGCGGGAGACTATCACTATCCCAGACCAG
ACAGACTCGGGTCCCTGTTGGCTCCATTTAAGTAGCAAGAGGACGTTATCGAACTGCGTACCGTTCTACACTGGGAGTTCAACGGTCCAGTACTGCAGCCGATGTAGTGCACCATGCGTAAGCGTGTGAAGCGTGCCTCGCGATTTACTGGCCGAT"""
Dna = Dna.split('\n')
k = 12
t = len(Dna)
print(t)
motifs, score = GreedyMotifSearch(Dna, k, t)
print(motifs)
print(score)

25
CCCTGTTGTGGA
CCCTGTTGGCTA
CCCTGTTGAACC
CCCTGTTGGAGT
CCCTGTTGTCAC
CCCTGTTGCCTA
CCCTGTTGGGTC
CCCTGTTGATAC
CCCTGTTGTATC
CCCTGTTGTCTA
CCCTGTTGCCAG
CCCTGTTGAACC
CCCTGTTGTTCT
CCCTGTTGTGGA
CCCTGTTGTATA
CCCTGTTGCGAG
CCCTGTTGCTGC
CCCTGTTGCATC
CCCTGTTGAGGT
CCCTGTTGAACC
CCCTGTTGCTTA
CCCTGTTGCTGG
CCCTGTTGGTAG
CCCTGTTGCTCC
CCCTGTTGGCTC
65


In [347]:
kmers = ['AGGTCA',
'ACGCGA',
'AAGCCA',
'AAGTGA',
'ACGTTA',
'AAGAGA']

profile = {
    'A': [0.4, 0.3, 0.0, 0.1, 0.0, 0.9],
    'C': [0.2, 0.3, 0.0, 0.4, 0.0, 0.1],
    'G': [0.1, 0.3, 1.0, 0.1, 0.5, 0.0],
    'T': [0.3, 0.1, 0.0, 0.4, 0.5, 0.0]
}

def getProb(km, profile):
    p = 1
    for i, c in enumerate(km):
        p = p*profile[c][i]
    return p

for km in kmers:
    print(getProb(km, profile))

0.0
0.0216
0.0
0.0216
0.0216
0.0054


In [386]:
def getProb(km, profile):
    p = 1
    for i, c in enumerate(km):
        p = p*profile[c][i]
    return p
profile = {
'A': [0.4, 0.3, 0.0, 0.1, 0.0, 0.9],
'C': [0.2, 0.3, 0.0, 0.4, 0.0, 0.1],
'G': [0.1, 0.3, 1.0, 0.1, 0.5, 0.0],
'T': [0.3, 0.1, 0.0, 0.4, 0.5, 0.0]
}
getProb('GAGCTA', profile)

0.0054